# Run db

In [5]:
!standalone.bat start # windown
!bash standalone_embed.sh start # linux

'standalone.bat' is not recognized as an internal or external command,
operable program or batch file.


# Test

In [21]:
# connect
from pymilvus import connections, db

conn = connections.connect(host="127.0.0.1", port=19530)

database = db.create_database("my_database")



2025-02-26 21:07:34,251 [ERROR][handler]: RPC error: [create_database], <MilvusException: (code=65535, message=database already exist: my_database)>, <Time:{'RPC start': '2025-02-26 21:07:34.247710', 'RPC error': '2025-02-26 21:07:34.251458'}> (decorators.py:140)


MilvusException: <MilvusException: (code=65535, message=database already exist: my_database)>

In [22]:
db.list_database()

['default', 'my_database']

In [ ]:
from pymilvus import MilvusClient, DataType
# connect
client = MilvusClient(
    uri="http://localhost:19530",
    token="root:Milvus"
)


# create schema
schema = MilvusClient.create_schema(
    auto_id=False,
    enable_dynamic_field=True,
)

schema.add_field(field_name="my_id", datatype=DataType.INT64, is_primary=True)
schema.add_field(field_name="my_vector", datatype=DataType.FLOAT_VECTOR, dim=5)
schema.add_field(field_name="my_varchar", datatype=DataType.VARCHAR, max_length=512)



{'auto_id': False, 'description': '', 'fields': [{'name': 'my_id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'my_vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 5}}, {'name': 'my_varchar', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 512}}], 'enable_dynamic_field': True}

In [24]:
# create index
index_params = client.prepare_index_params()
index_params.add_index(
    field_name="my_vector", 
    index_type="AUTOINDEX",
    metric_type="COSINE"
)

In [25]:
# create collection
client.create_collection(
    collection_name="customized_setup_1",
    schema=schema,
    index_params=index_params
)



2025-02-26 21:07:44,901 [ERROR][handler]: RPC error: [create_collection], <MilvusException: (code=65535, message=create duplicate collection with different parameters, collection: customized_setup_1)>, <Time:{'RPC start': '2025-02-26 21:07:44.896398', 'RPC error': '2025-02-26 21:07:44.901398'}> (decorators.py:140)
2025-02-26 21:07:44,902 [ERROR][_create_collection_with_schema]: Failed to create collection: customized_setup_1 (milvus_client.py:892)


MilvusException: <MilvusException: (code=65535, message=create duplicate collection with different parameters, collection: customized_setup_1)>

In [26]:
# describle collection
res = client.describe_collection(
    collection_name="customized_setup_1"
)

print(res)

{'collection_name': 'customized_setup_1', 'auto_id': False, 'num_shards': 1, 'description': '', 'fields': [{'field_id': 100, 'name': 'my_id', 'description': '', 'type': <DataType.INT64: 5>, 'params': {}, 'is_primary': True}, {'field_id': 101, 'name': 'my_vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 5}}, {'field_id': 102, 'name': 'my_varchar', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 512}}], 'functions': [], 'aliases': [], 'collection_id': 456281608620081432, 'consistency_level': 2, 'properties': {}, 'num_partitions': 1, 'enable_dynamic_field': True}


In [27]:
# releasse collection
client.release_collection(
    collection_name="customized_setup_1"
)

# Tạo db với vector nhị phân

In [28]:
schema = client.create_schema(
    auto_id=True,
    enable_dynamic_fields=True,
)

schema.add_field(field_name="pk", datatype=DataType.VARCHAR, is_primary=True, max_length=100)
schema.add_field(field_name="ingredients", datatype=DataType.BINARY_VECTOR, dim=128)
schema.add_field(field_name="recipes", datatype=DataType.VARCHAR, max_length=1000)

{'auto_id': True, 'description': '', 'fields': [{'name': 'pk', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 100}, 'is_primary': True, 'auto_id': False}, {'name': 'ingredients', 'description': '', 'type': <DataType.BINARY_VECTOR: 100>, 'params': {'dim': 128}}, {'name': 'recipes', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 1000}}], 'enable_dynamic_field': False}

In [29]:
index_params = client.prepare_index_params()

index_params.add_index(
    field_name="ingredients",
    index_name="ingredients_index",
    index_type="BIN_IVF_FLAT",
    metric_type="HAMMING",
    params={"nlist": 128} # nlist must be divided by 8 to convert into byte
)

In [30]:
client.create_collection(
    collection_name="cookbook_db",
    schema=schema,
    index_params=index_params
)

2025-02-26 21:07:52,016 [ERROR][handler]: RPC error: [create_collection], <MilvusException: (code=65535, message=create duplicate collection with different parameters, collection: cookbook_db)>, <Time:{'RPC start': '2025-02-26 21:07:52.011120', 'RPC error': '2025-02-26 21:07:52.016119'}> (decorators.py:140)
2025-02-26 21:07:52,017 [ERROR][_create_collection_with_schema]: Failed to create collection: cookbook_db (milvus_client.py:892)


MilvusException: <MilvusException: (code=65535, message=create duplicate collection with different parameters, collection: cookbook_db)>

In [31]:
def convert_bool_list_to_bytes(bool_list):
    if len(bool_list) % 8 != 0:
        raise ValueError("The length of a boolean list must be a multiple of 8")

    byte_array = bytearray(len(bool_list) // 8)
    for i, bit in enumerate(bool_list):
        if bit == 1:
            index = i // 8
            shift = i % 8
            byte_array[index] |= (1 << shift)
    return bytes(byte_array)

Data format: [{"col1" : "val1", "col2" : "val2"}, {}, ...]

In [32]:
# insert data

bool_vectors = [
    [1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0] + [0] * 112,
    [0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1] + [0] * 112,
]

data = [{"ingredients": convert_bool_list_to_bytes(bool_vector), "recipes": f"xin chao cac ban {i}" } for i, bool_vector in enumerate(bool_vectors)]


client.insert(
    collection_name="cookbook_db",
    data=data
)

{'insert_count': 2, 'ids': ['456282246184899109', '456282246184899110'], 'cost': 0}

In [33]:
# search
search_params = {
    "params": {"nprobe": 10}
}

query_bool_list = [1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0] + [0] * 112
query_vector = convert_bool_list_to_bytes(query_bool_list)

res = client.search(
    collection_name="cookbook_db",
    data=[query_vector],
    anns_field="ingredients",
    search_params=search_params,
    limit=1,
    output_fields=["pk", "recipes"]
)

print(res)

data: ["[{'id': '456281608620081615', 'distance': 0.0, 'entity': {'pk': '456281608620081615', 'recipes': 'xin chao cac ban 0'}}]"]
